## Task 2

The general approach taken for task 2 is as follows.

1) As in Task 1, we process the images using the pre-trained Vision Transformer (vit-base-patch16-224-in21k) and save the extracted features.
2) We load the prototypes and confidences generated after incorporating the last dataset (i.e D10) in Task 1.
3) For D11-D20, we use K-means clustering to generate 10 clusters, and each cluster is assigned to the closest prototype using cosine similarity. We update the prototypes using the cluster centroids in a similar way to what we did in Task 1.


In [2]:
import torch
from torchvision import transforms
from transformers import ViTModel, ViTFeatureExtractor
from PIL import Image
import os
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from collections import defaultdict
from sklearn.cluster import KMeans

We can now load the pre-made features for task 1 and task 2.

In [ ]:
# Train Data
data = torch.load("ExtractedFeatures/vit_features_train_1.pth")
features, targets = data['features'], data['targets']
X_train = normalize(features, axis=1)
Y_train = targets

# Eval Models
data = torch.load("ExtractedFeatures/vit_features_eval_1.pth")
features, targets = data['features'], data['targets']
X_test = normalize(features, axis=1)
Y_test = targets

# Number of classes in CIFAR-10
num_classes = 10

C:\Users\AADI RACIST\AppData\Local\Temp\ipykernel_11520\1335921280.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("ExtractedFeatures/ExtractedFeatures

In [4]:
# Compute class prototypes (mean feature vector for each class)
prototypes = np.load("prototypes.npy")
sizes = np.load("sizes.npy")

At each step we save cosine similarity with the closest mean to see how confident we are that this point belongs to this label.

In [5]:
# Classification function
def classify(sample, prototypes):
    similarities = cosine_similarity(sample.reshape(1, -1), prototypes)
    confidence = np.max(similarities)  # Confidence based on maximum similarity
    return np.argmax(similarities) , confidence

In [6]:
all_accuracies = []

We make 10 clusters for each dataset, and assign each cluster to the closest prototype. 

Now we take a weighted average using the cluster centroids to update the means. This ensures that wrong predictions are devalued while updating the means. 

In [7]:
def update_prototype(data_new , prototypes, sizes):
    X_train = normalize(data_new['features'], axis=1)
    preds = np.zeros(len(X_train))
    confidences = np.zeros(len(X_train))
    
    kmeans = KMeans(n_clusters=10, random_state=42)
    kmeans.fit(X_train)

    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_

    # Classify the centroids
    preds_c = np.zeros(len(centroids))
    confidences_c = np.zeros(len(centroids))

    for i in range(len(centroids)):
        preds_c[i], confidences_c[i] = classify(centroids[i], prototypes)

    # Classify the samples
    for i in range(len(X_train)):
        preds[i], confidences[i] = preds_c[labels[i]], confidences_c[labels[i]]

    # Update prototypes with weighted contributions
    for cls in range(num_classes):
        class_indices = np.where(preds == cls)[0]
        if len(class_indices) > 0:
            class_features = X_train[class_indices]
            class_confidences = confidences[class_indices].reshape(-1, 1)  # Reshape for broadcasting

            # Weighted sum of features
            weighted_sum = np.sum(class_features * class_confidences, axis=0)
            weighted_count = np.sum(class_confidences)  # Total confidence as a weight sum

            # Update the prototype
            prototypes[cls] = (
                prototypes[cls] * sizes[cls] + weighted_sum
            ) / (sizes[cls] + weighted_count)
            sizes[cls] += weighted_count  # Update size with the sum of confidences

    # Normalize prototypes for cosine similarity
    prototypes = normalize(prototypes, axis=1)
    return prototypes, sizes

In [ ]:
def holdout_accuracy(i , prototypes):
    accuracies = []
    for j in range(1,i+1):
        data = torch.load(f"/ExtractedFeatures/vit_features_eval_{j}.pth")
        features, targets = data['features'], data['targets']
        X_test = normalize(features, axis=1)
        Y_test = targets
        correct_predictions = 0
        for i in range(len(X_test)):
            prediction, _= classify(X_test[i], prototypes)
            if prediction == Y_test[i]:
                correct_predictions += 1

        accuracy = correct_predictions / len(X_test)
        accuracies.append(accuracy)
    return accuracies

In [9]:
for i in range(11,21):
    data = torch.load(f"ExtractedFeatures/vit_features_train_{i}.pth")
    prototypes, sizes = update_prototype(data, prototypes, sizes)

    accuracies = holdout_accuracy(i, prototypes)
    all_accuracies.append(accuracies)    


C:\Users\AADI RACIST\AppData\Local\Temp\ipykernel_11520\2850394873.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f"ExtractedFeatures/ExtractedFeature

In [10]:
all_accuracies

[[0.884,
  0.8908,
  0.904,
  0.9056,
  0.896,
  0.8948,
  0.8924,
  0.8948,
  0.8956,
  0.9016,
  0.7768],
 [0.8828,
  0.8892,
  0.9048,
  0.9044,
  0.8972,
  0.896,
  0.8936,
  0.8948,
  0.898,
  0.9004,
  0.7748,
  0.6528],
 [0.8852,
  0.8892,
  0.9036,
  0.904,
  0.8944,
  0.8936,
  0.8924,
  0.894,
  0.8924,
  0.8984,
  0.7696,
  0.6404,
  0.7752],
 [0.8832,
  0.888,
  0.9048,
  0.8996,
  0.894,
  0.8924,
  0.8936,
  0.8928,
  0.8924,
  0.8976,
  0.7688,
  0.638,
  0.7748,
  0.922],
 [0.8808,
  0.8884,
  0.904,
  0.9,
  0.8916,
  0.8908,
  0.8908,
  0.8916,
  0.8904,
  0.8988,
  0.7676,
  0.636,
  0.7764,
  0.9224,
  0.8828],
 [0.8812,
  0.8896,
  0.9028,
  0.8996,
  0.8936,
  0.8908,
  0.8904,
  0.8904,
  0.8928,
  0.9004,
  0.7704,
  0.6388,
  0.7744,
  0.9212,
  0.884,
  0.7624],
 [0.8808,
  0.8888,
  0.902,
  0.8976,
  0.8932,
  0.8916,
  0.8908,
  0.89,
  0.892,
  0.8984,
  0.7684,
  0.6356,
  0.7772,
  0.9208,
  0.8824,
  0.7628,
  0.8544],
 [0.878,
  0.8872,
  0.9012,
  0.8

Thus we that the loss of accuracies on older datasets are marginal, and some of them even improve, suggesting that confidence based updating allows us to get better ideas of the label means, improving our prototypes.